In [3]:
# importing the required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
class Node():
  def __init__(self, feature = None, threshold = None, left_child = None, right_child = None, leaf_value = None):
    self.feature = feature
    self.threshold = threshold
    self.left_child = left_child
    self.right_child = right_child
    self.leaf_value = leaf_value

In [6]:
class Tree():
  # constructor
  def __init__(self, X, n_min):
    self.root = None
    self.features = [feat for feat in range(X.shape[1])]
    self.n_min = round((n_min / 100) * X.shape[0])

  # build tree
  def grow_tree(self, X, y):
    labels = len(np.unique(y))
    
    # leaf conditions
    # 1. samples less than the n_min condition
    # 2. pure node
    if X.shape[0] < self.n_min or labels == 1:
      leaf_val = self.common_val(y) # finding the most common occurence
      return Node(leaf_value = leaf_val)
    
    # best feature based on max info gain
    best_feature, best_info_gain, best_thresh = self.best_feature(X, y)

    left_split, left_y, right_split, right_y = self.split_func(X, y, best_feature, best_thresh)

    # recursively creating sub trees
    left_subtree = self.grow_tree(left_split, left_y)
    right_subtree = self.grow_tree(right_split, right_y)
    return Node(best_feature, best_thresh, left_subtree, right_subtree)
 
  # average of adjacent vals
  def average_func(self, feature):
    mean_vals = np.zeros(len(feature))
    for i in range(len(feature)-1):
      mean_val = (feature[i] + feature[i+1]) / 2
      mean_vals[i] = mean_val
    return mean_vals

  # common value of a feature
  def common_val(self, y):
      vals = np.unique(y)
      counts = {}
      for val in vals:
        counts[val] = 0
      for x in y:
        counts[x] += 1
      max_val = max(counts, key = lambda x: counts[x]) 
      return max_val
    
  # entropy of the target column
  def entropy(self, y):
    unique_vals, counts = np.unique(y, return_counts = True)
    sum = 0
    for i in range(len(unique_vals)):
      prob_val = counts[i] / len(y)
      sum += - prob_val * np.log2(prob_val)
    return sum
  
  # split function
  def split_func(self, X, y, feature, threshold):
    left_split_feat = []
    left_split_target = []
    right_split_feat = []
    right_split_target = []
    for i in range(X.shape[0]):
      if X[i, feature] < threshold:
         left_split_feat.append(X[i, :])
         left_split_target.append(y[i])
      else:
         right_split_feat.append(X[i, :])
         right_split_target.append(y[i])
  
    return np.array(left_split_feat), np.array(left_split_target), np.array(right_split_feat), np.array(right_split_target)

  # information gain of a feature column
  def information_gain(self, feature_col, y, X, threshold):
    parent_entropy = self.entropy(y)
    left_split, left_y, right_split, right_y = self.split_func(X, y, feature_col, threshold)
    if len(left_split) == 0 or len(right_split) == 0:
      return 0
    left_child_entropy = ((len(left_split) / len(y)) * self.entropy(left_y))
    right_child_entropy = ((len(right_split) / len(y)) * self.entropy(right_y))
    gain = parent_entropy - (left_child_entropy + right_child_entropy)
    return gain
  
  # best feature based of max info gain
  def best_feature(self, X, y):
    n = X.shape[0] # number of samples
    m = X.shape[1] # number of features
    best_feature = None
    best_info_gain = -float("inf")
    best_thresh = None
    for feature in self.features:
      feature_values = X[:, feature]
      unique_feature_values = np.unique(feature_values)
      possible_thresholds = self.average_func(unique_feature_values)
      for threshold in possible_thresholds:
        info_gain = self.information_gain(feature, y, X, threshold)
        if info_gain > best_info_gain:
          best_info_gain = info_gain
          best_feature = feature
          best_thresh = threshold

    return best_feature, best_info_gain, best_thresh

  # predict
  def predict(self, X):
    preds = []
    for i in range(len(X)):
      curr_node = self.root
      while (curr_node.leaf_value == None):
        if (X[i, curr_node.feature] < curr_node.threshold):
          curr_node = curr_node.left_child
        else:
          curr_node = curr_node.right_child
      preds.append(curr_node.leaf_value)
    return preds
  
  # start building the tree
  def fit(self, X, y):
    self.root = self.grow_tree(X, y)

In [7]:
def accuracy(actual, pred):
  correct = 0
  for i in range(len(actual)):
    if actual[i] == pred[i]:
      correct += 1
  return (correct / len(actual))

In [8]:
def train_test(X, y, n_min):
  accuracy_list = [] # used to store accuracy for each fold
  kf = KFold(n_splits=10) # 10 fold cross validation
  for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    decision_tree = Tree(X_train, n_min)
    decision_tree.fit(X_train, y_train)
    y_pred = decision_tree.predict(X_test)
    accuracy_score = accuracy(y_test, np.array(y_pred))
    accuracy_list.append(accuracy_score)
  
  accuracy_list = np.array(accuracy_list)
  average_accuracy = np.mean(accuracy_list) # average
  standard_dev = np.std(accuracy_list) # standard deviation

  return average_accuracy, standard_dev

#### **Iris Datset**

In [9]:
iris_data = pd.read_csv("/content/drive/MyDrive/Applied ML/Lab-3/iris.csv", header = None)
# changing the labels of the target columns
iris_data[4] = iris_data[4].map({'Iris-setosa': 0,
              'Iris-versicolor': 1,
              'Iris-virginica': 2}) 

In [10]:
# separating features and targets
X = iris_data.iloc[:,:-1].values
y = iris_data.iloc[:,-1].values

In [11]:
n_mins_iris = [5, 10, 15, 20]

In [12]:
# dataframe to record accuracy and standard deviation
iris_results = pd.DataFrame(columns = ['N Min', 'Avg Accuracy', 'Standard Deviation'])

In [13]:
# training and testing with each n_min
for val in n_mins_iris:
  acc, sd = train_test(X, y, val)
  iris_results = iris_results.append({'N Min': val, 'Avg Accuracy': acc, 'Standard Deviation': sd}, ignore_index=True)

In [14]:
print(iris_results)

   N Min  Avg Accuracy  Standard Deviation
0    5.0      0.933333            0.078881
1   10.0      0.933333            0.078881
2   15.0      0.933333            0.078881
3   20.0      0.933333            0.078881


**Iris Results**


1.   n_min = 5
  *  Avg Accuracy = 0.933333 
  * Standard Deviation = 0.078881

2.   n_min = 10
  *  Avg Accuracy = 0.933333 
  * Standard Deviation = 0.078881
            
3.   n_min = 15
  *  Avg Accuracy = 0.933333 
  * Standard Deviation = 0.078881
            
4.   n_min = 20
  *  Avg Accuracy = 0.933333 
  * Standard Deviation = 0.078881
           
5.   n_min = 25
  *  Avg Accuracy = 0.933333 
  * Standard Deviation = 0.078881

#### **Spam Base**

In [15]:
spam_data = pd.read_csv("/content/drive/MyDrive/Applied ML/Lab-3/spambase.csv", header = None)
# separating features and targets
X2 = spam_data.iloc[:,:-1].values
y2= spam_data.iloc[:,-1].values

In [16]:
n_mins_spam = [5, 10, 15, 20, 25]

In [17]:
# dataframe to store accuracy and standard deviation for each n_min
spam_results = pd.DataFrame(columns = ['N Min', 'Avg Accuracy', 'Standard Deviation'])

In [18]:
for val in n_mins_spam:
  acc, sd = train_test(X2, y2, val)
  spam_results = spam_results.append({'N Min': val, 'Avg Accuracy': acc, 'Standard Deviation': sd}, ignore_index=True)
  print(spam_results)

   N Min  Avg Accuracy  Standard Deviation
0    5.0       0.87373            0.062572
   N Min  Avg Accuracy  Standard Deviation
0    5.0      0.873730            0.062572
1   10.0      0.871562            0.066349
   N Min  Avg Accuracy  Standard Deviation
0    5.0      0.873730            0.062572
1   10.0      0.871562            0.066349
2   15.0      0.840285            0.074703
   N Min  Avg Accuracy  Standard Deviation
0    5.0      0.873730            0.062572
1   10.0      0.871562            0.066349
2   15.0      0.840285            0.074703
3   20.0      0.823764            0.102280
   N Min  Avg Accuracy  Standard Deviation
0    5.0      0.873730            0.062572
1   10.0      0.871562            0.066349
2   15.0      0.840285            0.074703
3   20.0      0.823764            0.102280
4   25.0      0.783327            0.101522


In [19]:
print(spam_results)

   N Min  Avg Accuracy  Standard Deviation
0    5.0      0.873730            0.062572
1   10.0      0.871562            0.066349
2   15.0      0.840285            0.074703
3   20.0      0.823764            0.102280
4   25.0      0.783327            0.101522


**Spambase Results**


1.   n_min = 5
  *  Avg Accuracy = 0.873730
  * Standard Deviation = 0.062572

2.   n_min = 10
  * Average Accuracy = 0.871562
  * Standard Deviation = 0.066349
            
3.   n_min = 15
  * Average Accuracy = 0.840285
  * Standard Deviation = 0.074703
            
4.   n_min = 20
  * Average Accuracy = 0.823764 
  * Standard Deviation = 0.102280
           
5.   n_min = 25
  * Average Accuracy = 0.783327 
  * Standard Deviation = 0.101522
           



